In [1]:
import os
import sys
import src

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/goldyrana/work/dl/kidney_scans/end_to_end_dl_project_kidney_scans'

In [4]:
import zipfile
import gdown
from pathlib import Path
from dataclasses import dataclass
from typing import Any
from box import ConfigBox
from src.common import read_yaml

In [6]:
@dataclass
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_agumentation: bool
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [13]:
from src.common import read_yaml, config_path, params_path

class ConfigurationManager:
    def __init__(self):
        self.config_path = config_path
        self.params_path = params_path
        
        self.config = read_yaml(self.config_path)
        self.params = read_yaml(self.params_path)
        
    def get_prepare_base_model_config(self) -> BaseModelConfig:
        config = self.config.prepare_base_model
        
        return BaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_agumentation=self.params.AUGMENTATION,
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
            )
        

        

In [22]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.models import Sequential
import urllib.request as request
from zipfile import ZipFile

In [23]:
class prepareBaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
            )
        self.save_model(self.config.base_model_path, self.model)
        
    @staticmethod
    def _prepare_full_model(model, classes, learning_rate):
        flatten_in = tf.keras.layers.Flatten()(model.output)
        full_model = Sequential([model,
                                 Flatten(),
                                 Dense(units=256, activation='relu'),
                                 Dense(units=classes, activation='softmax')])
    
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"])

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate
        )
        
        self.save_model(self.config.updated_base_model_path, model = self.full_model)

In [24]:
try:
    config = ConfigurationManager()
    base_model_config = config.get_prepare_base_model_config()
    obj = prepareBaseModel(base_model_config)
    obj.get_base_model()
    obj.update_base_model()
except Exception as e:
    raise e
    


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               6422784   
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21137986 (80.64 MB)
Trainable params: 21137986 (80.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
